In [2]:
# Part A (New Version for FLAME)

import torch
import numpy as np
import pickle # Use the pickle library for .pkl files
import matplotlib.pyplot as plt
import chumpy # Explicitly import chumpy to test availability

# --- 1. Define the path to your FLAME model file ---
# This should point to the .pkl model file (e.g., flame2023.pkl)
flame_model_path = '../data/flame_model/flame2023.pkl' 

flame_model = None      # Initialize to None
mean_shape = None       # Initialize
triangles = None        # Initialize
num_vertices = 0      # Initialize
num_triangles = 0     # Initialize

try:
    # Load the model using pickle for .pkl files
    with open(flame_model_path, 'rb') as f:
        flame_model = pickle.load(f, encoding='latin1')
    print(f"FLAME 2023 model loaded successfully from {flame_model_path}.")
except FileNotFoundError:
    print(f"ERROR: FLAME model not found at {flame_model_path}. Please check the path and README instructions.")
except Exception as e: # Catch other potential errors during pickle loading, e.g., chumpy issues if not installed
    print(f"ERROR: Could not load FLAME model from {flame_model_path}. Error: {e}. Ensure 'chumpy' is installed if using .pkl files.")


# --- 2. Extract and Prepare Key Components ---
# Ensure the model data was loaded before proceeding
if flame_model is not None:
    # v_template is the average face shape (the equivalent of BFM's shapeMU)
    # For FLAME .pkl, 'v_template' is often a chumpy object, need .r to get numpy array
    if hasattr(flame_model['v_template'], 'r'): # Check if it's a chumpy object with .r attribute
        shape_data = flame_model['v_template'].r
    else:
        shape_data = flame_model['v_template'] # Assume it's already a numpy array
    mean_shape = torch.from_numpy(shape_data).float()

    # f contains the face triangles (the equivalent of BFM's tl)
    # It's already 0-indexed, so we don't need to subtract 1
    # Ensure 'f' is of integer type for PyTorch3D faces
    triangles = torch.from_numpy(flame_model['f'].astype(np.int64))

    # Get the number of vertices and faces
    num_vertices = mean_shape.shape[0]
    num_triangles = triangles.shape[0]
    print(f"Data parsed: {num_vertices} vertices, {num_triangles} triangles.")
else:
    # Handle the case where model loading failed by creating dummy tensors
    # This allows the rest of the notebook to run without crashing, though rendering will be meaningless.
    print("FLAME model data not loaded. Using placeholder data for subsequent cells.")
    mean_shape = torch.zeros((10, 3), dtype=torch.float32) # Dummy shape
    triangles = torch.zeros((5, 3), dtype=torch.int64)    # Dummy triangles
    num_vertices = mean_shape.shape[0]
    num_triangles = triangles.shape[0]

# --- 3. Create a Generic Texture ---
# The base FLAME model doesn't come with a mean texture map like BFM.
# We will create a simple, uniform gray color for now.
# The goal is to confirm the SHAPE is rendering correctly.
# We create one color (e.g., gray) and repeat it for all vertices.
generic_color = torch.tensor([0.7, 0.7, 0.7]) # A nice medium gray
vertex_colors = generic_color.view(1, 3).expand(num_vertices, 3)

/tmp/ipykernel_2010964/1408266182.py:15: DeprecationWarning: Please import `csc_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csc` namespace is deprecated and will be removed in SciPy 2.0.0.
  flame_model = pickle.load(f, encoding='latin1')


ModuleNotFoundError: No module named 'chumpy'

In [ ]:
# Part B (New Version for FLAME)

from pytorch3d.structures import Meshes

# We use our new variables to create the mesh.
# The texture is now our generic gray color per vertex.
# We need to add a batch dimension for PyTorch3D -> (1, num_vertices, 3)
verts_rgb = vertex_colors.unsqueeze(0)
textures = verts_rgb

# Create the Meshes object
average_face_mesh = Meshes(
    verts=[mean_shape], # Use the mean_shape directly
    faces=[triangles],
    textures=textures
)
print("PyTorch3D Meshes object created with FLAME 2023 model.")

In [1]:
import torch # Ensure torch is imported in this cell
import matplotlib.pyplot as plt # Ensure plt is imported in this cell
from pytorch3d.renderer import (
    look_at_view_transform, FoVPerspectiveCameras, PointLights, RasterizationSettings,
    MeshRenderer, MeshRasterizer, SoftPhongShader
)

# --- Select a device ---
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# --- Set up the renderer (camera, lights, etc.) ---
R, T = look_at_view_transform(dist=2.7, elev=0, azim=0)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
raster_settings = RasterizationSettings(image_size=512, blur_radius=0.0, faces_per_pixel=1)
lights = PointLights(device=device, location=[[0.0, 0.0, 3.0]])
shader = SoftPhongShader(device=device, cameras=cameras, lights=lights)

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=shader
)

# --- Move mesh to the correct device and render! ---
average_face_mesh = average_face_mesh.to(device)
images = renderer(average_face_mesh)

# --- Visualize the output ---
plt.figure(figsize=(8, 8))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off")
plt.title("Success! The Rendered Average Face")
plt.show()

NameError: name 'torch' is not defined